In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk import ngrams, FreqDist
from nltk import ngrams, FreqDist
from collections import Counter

# Import Portuguese NLP Package
#import sys
#sys.path.append('C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\NLP\\NLPyPort-master')
from NLPyPort.FullPipeline import *

pd.options.display.max_columns = None
pd.set_option("display.max_rows", None)

ModuleNotFoundError: No module named 'NLPyPort.LemPyPort'

# Utils

# Clean Up DataFrame

In [2]:
df_cml = pd.read_csv('CML\\artworks.csv',encoding='utf-16',index_col=0)
df_cml = df_cml.reset_index().drop('index',axis=1)

In [3]:
df_cml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4552 entries, 0 to 3995
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Auction Number       4552 non-null   int64  
 1   Artwork Number       4552 non-null   int64  
 2   Author               3213 non-null   object 
 3   Title                4552 non-null   object 
 4   Technique            4549 non-null   object 
 5   Dimensions           4518 non-null   object 
 6   Estimated Price      4552 non-null   int64  
 7   Date of Auction End  4552 non-null   object 
 8   Sale Price           4552 non-null   object 
 9   Image URL            4552 non-null   object 
 10  Unnamed: 0.1         3996 non-null   float64
dtypes: float64(1), int64(3), object(7)
memory usage: 426.8+ KB


Let's explore the date.

In [4]:
df_cml.head()

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
0,1267,1,MÁRIO CESARINY - 1923-2006,"""Ready-made absoluto (não interferido)""","técnica mista sobre papel, assinada e datada d...",Dim. - 34 x 27 cm,600,"Oct 25, 2020 9:00:00 PM",600,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
1,1267,2,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre impressão, assinado","Dim. - 19,5 x 14 cm",200,"Oct 25, 2020 9:00:15 PM",260,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
2,1267,3,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 18 x 23,5 cm",700,"Oct 25, 2020 9:00:30 PM",725,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
3,1267,4,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22,5 x 18 cm",600,"Oct 25, 2020 9:00:45 PM",650,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
4,1267,5,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22 x 18,5 cm",600,"Oct 25, 2020 9:01:00 PM",850,https://leiloescml-aclserviatildese.netdna-ssl...,NaN


## Dealing with Missing Values

### Author

Let's look for missing valus in the "Author" column:

In [5]:
print('There are {} missing values.'.format(df_cml['Author'].isna().sum()))

There are 1339 missing values.


How many of these are because no Author exists and how many are due to this information being in one of the other columns due to scraping or because author was included in another part of the URL HTML?

In [6]:
df_cml[df_cml['Author'].isna()].sample(10)

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
3614,1025,19,NaN,Paisagem com casa,"aguarela sobre papel, portuguesa, séc. XX, ass...","Dim. - 19,5 x 21,5 cm",40,"Apr 23, 2017 9:06:56 PM",42,https://leiloescml-aclserviatildese.netdna-ssl...,3614.0
3068,1066,202,NaN,Mesa de apoio/Candeeiro,"estrutura em metal, tampo de fórmica, pés em m...",Dim. - 170 x 50 x 39 cm,80,"Dec 17, 2017 9:50:15 PM",80,https://leiloescml-aclserviatildese.netdna-ssl...,3068.0
3950,1014,144,NaN,Centro de mesa,"cristal moldado, decoração ""Motivos marinhos"",...",Dim. - 7 x 32 cm,40,"Feb 5, 2017 9:35:45 PM",40,https://leiloescml-aclserviatildese.netdna-ssl...,3950.0
1715,1170,295,NaN,Taça «canoa»,"faiança, decoração policromada ""Gato"", europei...","Dim. - 5,5 x 40,6 x 13,5 cm",50,"May 19, 2019 10:13:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1715.0
3964,1014,158,NaN,Par de estantes,"metal dourado, prateleiras em vidro fumado, po...",Dim. - 153 x 39 x 39 cm,80,"Feb 5, 2017 9:44:28 PM",150,https://leiloescml-aclserviatildese.netdna-ssl...,3964.0
390,1247,214,NaN,Figura feminina,escultura em madeira com tachas e fio de cobre...,Dim. - 27 cm,270,"Jun 28, 2020 9:53:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
2019,1142,299,NaN,Músicos,"escultura em diversos materiais, escola portug...",Dim. - 58 x 60 cm,40,"Dec 30, 2018 10:15:24 PM",75,https://leiloescml-aclserviatildese.netdna-ssl...,2019.0
1383,1183,309,NaN,Sem título - Torsos,"escultura em bronze, base em mármore, portugue...",Dim. - (escultura) 13 cm; (total) 23 cm,30,"Jul 28, 2019 10:17:00 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1383.0
2658,1084,71,NaN,Jarra,"vidro colorido, europeia, séc. XX","Dim. - 35,5 cm",20,"Mar 19, 2018 9:17:30 PM",20,https://leiloescml-aclserviatildese.netdna-ssl...,2658.0
1368,1183,294,NaN,Sem título,"serigrafia sobre papel, assinada Américo Silva...",Dim. - 50 x 50 cm,50,"Jul 28, 2019 10:13:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1368.0


Most cases appear to have correct information on the title. I have however found cases where the Author appears on the Title itself as this case for example:

In [7]:
df_cml.iloc[1035]

Auction Number                                                      1218
Artwork Number                                                        23
Author                                           ELSA CÉSAR - NASC. 1947
Title                                                         Sem título
Technique              técnica mista com colagem sobre papel, assinad...
Dimensions                                             Dim. - 32 x 25 cm
Estimated Price                                                       20
Date of Auction End                              Jan 12, 2020 9:05:30 PM
Sale Price                                                            22
Image URL              https://leiloescml-aclserviatildese.netdna-ssl...
Unnamed: 0.1                                                         479
Name: 479, dtype: object

So if no author can be found elsewhere, we can change the NaN by a "Sem Autor". 

However, it may be possible to recover the information on another field. In some instances, the name was captured in the "title" field. Usually a name is accompanied by a date in (). So we will use REGEX to identify a pattern of "(\d".  

If no name can be found on any other field, the value should be set to "Sem Informação". 

In [8]:
filt = (df_cml['Title'].str.contains('\(\d')) & (df_cml['Author'].isna()) & (~df_cml['Title'].str.lower().str.contains('álbum'))
df_cml[filt]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
1035,1199,261,NaN,Jean Baptiste Valadié (1933) - figuras femininas,"Serigrafia sobre papel Japão, E.A. II/XXV, ass...",Dim. - 65 x 50 cm,20,"Nov 3, 2019 10:05:00 PM",22,https://leiloescml-aclserviatildese.netdna-ssl...,1035.0
1391,1183,317,NaN,Alvar Aalto (1898 - 1976) Jarra,"Modernista, vidro azul, moldado, Finlândia, sé...",Dim. - 12 x 15 x 15 cm,40,"Jul 28, 2019 10:21:50 PM",110,https://leiloescml-aclserviatildese.netdna-ssl...,1391.0
3477,1034,69,NaN,Neville Chamberlain (1869-1940) - caricatura,"escultura em barro pintado, cabeça partida e c...",Dim. - 52 cm,200,"Jun 25, 2017 9:17:00 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,3477.0


Few hits and one is Neville Chamberlain, (in)famous British Prime Minister of the start of WW2!Let's exclude "caricatura" from the search.

We must extricate the author and dates and leave the title itself intact. Let's do it.

In this case, we can extract everything that comes before the ).

In [9]:
filt = (df_cml['Title'].str.contains('\(\d')) & (df_cml['Author'].isna()) & (~df_cml['Title'].str.lower().str.contains('álbum')) & (~df_cml['Title'].str.lower().str.contains('caricatura'))
df_cml[filt]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
1035,1199,261,NaN,Jean Baptiste Valadié (1933) - figuras femininas,"Serigrafia sobre papel Japão, E.A. II/XXV, ass...",Dim. - 65 x 50 cm,20,"Nov 3, 2019 10:05:00 PM",22,https://leiloescml-aclserviatildese.netdna-ssl...,1035.0
1391,1183,317,NaN,Alvar Aalto (1898 - 1976) Jarra,"Modernista, vidro azul, moldado, Finlândia, sé...",Dim. - 12 x 15 x 15 cm,40,"Jul 28, 2019 10:21:50 PM",110,https://leiloescml-aclserviatildese.netdna-ssl...,1391.0


We should also make surethe name and dates are in the format of the remaining ones in Author "Name - Dates".

In [10]:
#Transport the author to the Author column
df_cml.loc[filt,'Author'] = df_cml[filt]['Title'].str.extract(r'(.*?\))')[0].str.replace('(','- ').str.replace(')','')

In [11]:
#Removet the Author from the Title column
df_cml.loc[filt,'Title'] = df_cml[filt]['Title'].str.extract(r'\)(.*)')[0].str.replace('- ','')

### Replace the NaN by "No information"

In [12]:
df_cml['Author'] = df_cml['Author'].fillna('No information')

In [13]:
df_cml['Author'].isna().sum()

0

#### Portuguese Names

In [14]:
df_names_PT = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\nomes-registados-2016.csv',usecols=[0],header=None,names=['Name_PT'])
df_names_PT = df_names_PT.dropna()
df_names_PT.head()

FileNotFoundError: [Errno 2] File C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\nomes-registados-2016.csv does not exist: 'C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\Arte\\nomes-registados-2016.csv'

#### English Names

In [15]:
df_names_EN = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\baby-names.csv',usecols=[1],header=None,names=['Name_EN']).drop(0).reset_index(drop=True)
df_names_EN = df_names_EN.dropna()
df_names_EN.head()

FileNotFoundError: [Errno 2] File C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\baby-names.csv does not exist: 'C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\Arte\\baby-names.csv'

#### French Names

In [16]:
df_names_FR = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\Prenoms.csv',sep=';',usecols=[0],header=None,names=['Name_FR'],encoding='latin-1').drop(0).reset_index(drop=True)
df_names_FR = df_names_FR.dropna()
df_names_FR.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\Arte\\Prenoms.csv'

#### Spanish Names

In [17]:
df_names_ES_male = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\hombres.csv',usecols=[0],header=None,names=['Name_ES']).drop(0).reset_index(drop=True)
df_names_ES_female = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\mujeres.csv',usecols=[0],header=None,names=['Name_ES']).drop(0).reset_index(drop=True)

#Merge the male and female datasets
df_names_ES = pd.concat((df_names_ES_male,df_names_ES_female),axis=0)
df_names_ES = df_names_ES.dropna()
del df_names_ES_male,df_names_ES_female

FileNotFoundError: [Errno 2] File C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\hombres.csv does not exist: 'C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\Arte\\hombres.csv'

In [18]:
df_names_ES.sample(5)

NameError: name 'df_names_ES' is not defined

#### German Names

In [19]:
df_names_DE = pd.read_csv(r'C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\Vornamen_Koeln_2017.csv',usecols=[0],header=None,names=['Name_GE']).drop(0).reset_index(drop=True)
df_names_DE = df_names_DE.dropna()
df_names_DE.head()

FileNotFoundError: [Errno 2] File C:\Users\miguel.almas\OneDrive - insidemedia.net\DataScience\Arte\Vornamen_Koeln_2017.csv does not exist: 'C:\\Users\\miguel.almas\\OneDrive - insidemedia.net\\DataScience\\Arte\\Vornamen_Koeln_2017.csv'

### Look for Names on the DataFrame's column

In [20]:
aux = df_cml[(df_cml['Author'].isna())]
aux[aux['Title'].str.contains('|'.join(df_names_PT['Name_PT'].str.lower().tolist()+
                                       df_names_DE['Name_GE'].str.lower().tolist()+
                                       df_names_EN['Name_EN'].str.lower().tolist()+
                                       df_names_FR['Name_FR'].str.lower().tolist()+
                                       df_names_ES['Name_ES'].str.lower().tolist()))]

NameError: name 'df_names_PT' is not defined

## Technique

In [21]:
df_cml[df_cml['Technique'].isna()]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
1402,1183,328,No information,Telefonia,NaN,NaN,20,"Jul 28, 2019 10:21:45 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1402.0
1979,1142,259,No information,"Conjunto de material fotográfico composto por,...",NaN,Dim. - 9 x 15 x 9 cm,50,"Dec 30, 2018 10:07:13 PM",50,https://leiloescml-aclserviatildese.netdna-ssl...,1979.0
1980,1142,260,No information,Câmara fotográfica ZENIT,NaN,"Dim. - 9 x 13,5 x 11 cm",40,"Dec 30, 2018 10:04:45 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1980.0


These appear to be less of an art piece and more of an old equipment! We'll exlude these cases.

In [22]:
#Remove the entries without Technique information
df_cml = df_cml[df_cml['Technique'].isna() == False].copy()

## Dimensions

In [23]:
df_cml[df_cml['Dimensions'].isna()]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
370,1247,194,EDUARDO LUIZ - 1932-1988,Sem título,"serigrafia sobre papel, assinada e datada de 1...",NaN,150,"Jun 28, 2020 9:50:16 PM",150,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
77,1222,78,MANUEL CARGALEIRO - NASC. 1927,Sem título,"serigrafia sobre papel, assinada e datada de 1...",NaN,200,"Feb 2, 2020 9:19:15 PM",575,https://leiloescml-aclserviatildese.netdna-ssl...,77.0
328,1222,329,GRAÇA MORAIS - NASC. 1948,Sem título,"técnica mista sobre papel, assinada, dedicada ...",NaN,400,"Feb 2, 2020 10:22:00 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,328.0
418,1222,419,No information,Sem título,"gravura sobre papel, picos de acidez, defeito,...",NaN,30,"Feb 2, 2020 10:44:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,418.0
454,1222,455,PEDRO CHARTERS D'AZEVEDO - NASC. 1946,Sem título,"técnica mista sobre tela, assinado",NaN,600,"Feb 2, 2020 10:53:30 PM",600,https://leiloescml-aclserviatildese.netdna-ssl...,454.0
1097,1183,23,PEDRO PROENÇA - NASC. 1962,Sem título,"serigrafia sobre papel, pequeno defeito no sup...",NaN,80,"Jul 28, 2019 9:05:30 PM",110,https://leiloescml-aclserviatildese.netdna-ssl...,1097.0
1148,1183,74,SIZA VIEIRA - NASC. 1933,Sem título,"dois azulejos da Fábrica Viúva Lamego, assinados",NaN,60,"Jul 28, 2019 9:18:15 PM",70,https://leiloescml-aclserviatildese.netdna-ssl...,1148.0
1248,1183,174,No information,Serviço de café para seis pessoas,"barro pintado e vidrado, composto por bule e s...",NaN,50,"Jul 28, 2019 9:43:15 PM",50,https://leiloescml-aclserviatildese.netdna-ssl...,1248.0
1273,1183,199,LUIZ PIZARRO – NASC. 1958,Sem título,"pastel sobre papel, assinado no verso",NaN,300,"Jul 28, 2019 9:49:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1273.0
1717,1170,297,No information,PENCK. AR. - Mein Protokoll,"PENCK. AR.-Mein Protokoll, Bitterfelder Konfer...",NaN,100,"May 19, 2019 10:14:00 PM",100,https://leiloescml-aclserviatildese.netdna-ssl...,1717.0


We can drops these lines or try to infer them. 

Most refer to things that are not paintings though not all which incentivises the drop option.

We can also try to infer them, but for that we should be able to use other columns information, but pre-processing is necessary before hand.

Let's leave them as they are for now!

In [24]:
for i in df_cml.columns:
    print('For column {} we have {} missing values.'.format(i,df_cml[i].isna().sum()))

For column Auction Number we have 0 missing values.
For column Artwork Number we have 0 missing values.
For column Author we have 0 missing values.
For column Title we have 0 missing values.
For column Technique we have 0 missing values.
For column Dimensions we have 33 missing values.
For column Estimated Price we have 0 missing values.
For column Date of Auction End we have 0 missing values.
For column Sale Price we have 0 missing values.
For column Image URL we have 0 missing values.
For column Unnamed: 0.1 we have 556 missing values.


# Pre-Processing - Feature Extraction

## Author

In [25]:
df_cml.head()

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1
0,1267,1,MÁRIO CESARINY - 1923-2006,"""Ready-made absoluto (não interferido)""","técnica mista sobre papel, assinada e datada d...",Dim. - 34 x 27 cm,600,"Oct 25, 2020 9:00:00 PM",600,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
1,1267,2,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre impressão, assinado","Dim. - 19,5 x 14 cm",200,"Oct 25, 2020 9:00:15 PM",260,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
2,1267,3,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 18 x 23,5 cm",700,"Oct 25, 2020 9:00:30 PM",725,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
3,1267,4,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22,5 x 18 cm",600,"Oct 25, 2020 9:00:45 PM",650,https://leiloescml-aclserviatildese.netdna-ssl...,NaN
4,1267,5,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22 x 18,5 cm",600,"Oct 25, 2020 9:01:00 PM",850,https://leiloescml-aclserviatildese.netdna-ssl...,NaN


So Author field contains the name and surnames of the artist as well as the date of birth/death.

We want to divide these into 3 columns, "Author Name", "Author Birth", "Author Death". And of course not all cases will have date of death of even date of birth! Dates can also be a number or a century in which case it may be in roman numerals (XX, XIX, etc.).

#### Name

In [26]:
df_cml['Author'].head(285).tail(10)

275    JOSÉ PEDRO CROFT - NASC. 1957
276    JOSÉ PEDRO CROFT - NASC. 1957
277    JOSÉ PEDRO CROFT - NASC. 1957
278    JOSÉ PEDRO CROFT - NASC. 1957
279        BERTINA LOPES - 1924-2012
280       NUNO SANTIAGO - NASC. 1956
281        JOÃO MURILLO - NASC. 1967
282        NÁDIA DUVALL - NASC. 1986
283       MICHAEL GRAVES - 1962-2015
284      ISABEL LAGINHAS - 1942-2018
Name: Author, dtype: object

The separation of name and dates seems to be a ' - '. However, some authors also seem to have this in the same (TOM - THOMAZ DE MELLO being an example). So perhaps we want to capture everything that appears before the last ' - '?

In [27]:
def clean_author_name(series_author):
    """
    This small function receives a Pandas Series with the Author name and cleans it.
    """
    #Clean the Author Name
    df_author_name_to_clean = series_author.str.lower().str.replace('(nasc. [0-9]+)|(séc. xix/xx)|(séc. xx/xxi)|(séc. xxi)|(séc. xx)|(séc. xix)|(nasc. xx)|(nasc. xix)|(nasc. xxi)|(nasc. xxi)|([0-9]+)','')
    series_author = df_author_name_to_clean.str.replace('(- )+$|(-)+|( - )+|( -)+$','').str.rstrip().str.rstrip(',')
    series_author = series_author.str.replace(' ()','',regex=False)

    #Delete the temporary dataframe
    del df_author_name_to_clean

    return series_author

In [28]:
#Call function to clean Author Name
df_cml['Author Name'] = clean_author_name(df_cml['Author'])

#### Number of Authors

In [29]:
df_cml['Number of Authors'] = df_cml['Author Name'].str.split('/|,').apply(lambda x: len(x))

In [30]:
df_cml['Number of Authors'].value_counts()

1    4540
2       8
3       1
Name: Number of Authors, dtype: int64

The may be more than 1 author per artwork. Let's acoomodate that into multiple columns of artist names!

In [31]:
#Create columns with the name of authors (1 author, 2 author, etc.)
for i in range(df_cml['Number of Authors'].max()):
    df_cml[str(i+1)+' Author Name'] = 'none'
    df_cml.loc[df_cml['Number of Authors'] >= i+1,str(i+1)+' Author Name'] = (df_cml[df_cml['Number of Authors'] >= i+1]['Author Name'].str.split('/|,')).apply(lambda x: x[i]).str.lstrip()

In [32]:
#Drop the "Author Name Column"
df_cml = df_cml.drop('Author Name',axis=1)

It seems that it is done!

#### Dates

Onwards to the dates.

In [45]:
#Extract dates from various lines
df_dates = df_cml['Author'].str.lower().str.extractall('(nasc. [0-9]+)|(séc. xxi)|(séc. xx)|(séc. xix)|(nasc. xx)|(nasc. xix)|(nasc. xx/xxi)|(nasc. xxi)|(nasc. xxi)|([0-9]+)|(no information)')
df_dates.head(10)

0    1    2    3    4    5    6    7    8     9    10
  match                                                               
0 0             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1923  NaN
  1             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2006  NaN
1 0      nasc. 1920  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN
2 0      nasc. 1920  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN
3 0      nasc. 1920  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN
4 0      nasc. 1920  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN
5 0             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1894  NaN
  1             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1971  NaN
6 0             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1894  NaN
  1             NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1971  NaN

We have extracted all date information from the DataFrame and must now convert it into 2 columns, date(s) of birth, date(s) of death.

In [34]:
#Merge all columns into a single one
df_dates = pd.DataFrame(df_dates.apply(lambda x: ','.join(x.dropna().astype(str)),axis=1))
#Obtain index of rows that must be joined (cases where 2 or more dates exist for the same artwork)
index_nbr = df_dates[df_dates.index.get_level_values(1) == 1].index.get_level_values(0).tolist()

#For those indexes, join all dates in single row
for i in index_nbr:
    df_dates.loc[i,0] = df_dates.loc[i].apply(lambda x: ','.join(x.dropna().astype(str)),axis=0).values[0]
#Drop the the second level rows
df_dates = df_dates.xs(0, level=1)

C:\Users\miguel.almas\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


We can now try and obtain dates of death from the global dates and deal with cases with multiple authors.

In [40]:
df_dates[0]

0       1923,2006,nasc. 1953
1                 nasc. 1920
2                 nasc. 1920
3                 nasc. 1920
4                 nasc. 1920
                ...         
3991          no information
3992          no information
3993          no information
3994          no information
3995          no information
Name: 0, Length: 4548, dtype: object

In [38]:
df_cml.shape

(4549, 15)

In [57]:
df_dates.index.duplicated()

array([False, False, False, ..., False, False, False])

In [56]:
df_cml.index.duplicated()

array([False, False, False, ..., False, False, False])

In [58]:
df_cml

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1,Number of Authors,1 Author Name,2 Author Name,3 Author Name
0,1267,1,MÁRIO CESARINY - 1923-2006,"""Ready-made absoluto (não interferido)""","técnica mista sobre papel, assinada e datada d...",Dim. - 34 x 27 cm,600,"Oct 25, 2020 9:00:00 PM",600,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,mário cesariny,none,none
1,1267,2,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre impressão, assinado","Dim. - 19,5 x 14 cm",200,"Oct 25, 2020 9:00:15 PM",260,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
2,1267,3,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 18 x 23,5 cm",700,"Oct 25, 2020 9:00:30 PM",725,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
3,1267,4,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22,5 x 18 cm",600,"Oct 25, 2020 9:00:45 PM",650,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
4,1267,5,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22 x 18,5 cm",600,"Oct 25, 2020 9:01:00 PM",850,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,1014,185,No information,Rádio PHILCO,"americano, sinais de uso, faltas e defeitos, m...","Dim. - 51 x 38,5 x 26,5 cm",20,"Feb 5, 2017 9:46:00 PM",24,https://leiloescml-aclserviatildese.netdna-ssl...,3991.0,1,no information,none,none
3992,1014,186,No information,Rádio BUSH,"inglês, séc. XX, sinais de uso, mecanismo a ne...",Dim. - 23 x 32 x 19 cm,10,"Feb 5, 2017 9:46:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,3992.0,1,no information,none,none
3993,1014,187,No information,Rádio EMERSON,"modelo 268-LW, americano, sinais de uso, mecan...",Dim. - 19 x 30 x 17 cm,10,"Feb 5, 2017 9:46:30 PM",30,https://leiloescml-aclserviatildese.netdna-ssl...,3993.0,1,no information,none,none
3994,1014,188,No information,Rádio PHILIPS,"holandês, séc. XX, sinais de uso, mecanismo a ...",Dim. - 25 x 26 x 12 cm,10,"Feb 5, 2017 9:46:45 PM",20,https://leiloescml-aclserviatildese.netdna-ssl...,3994.0,1,no information,none,none


In [41]:
#Import to new column of Dates
df_cml['Dates'] = df_dates[0]

del df_dates

ValueError: cannot reindex from a duplicate axis

In [36]:
df_cml.head()

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Unnamed: 0.1,Number of Authors,1 Author Name,2 Author Name,3 Author Name
0,1267,1,MÁRIO CESARINY - 1923-2006,"""Ready-made absoluto (não interferido)""","técnica mista sobre papel, assinada e datada d...",Dim. - 34 x 27 cm,600,"Oct 25, 2020 9:00:00 PM",600,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,mário cesariny,none,none
1,1267,2,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre impressão, assinado","Dim. - 19,5 x 14 cm",200,"Oct 25, 2020 9:00:15 PM",260,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
2,1267,3,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 18 x 23,5 cm",700,"Oct 25, 2020 9:00:30 PM",725,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
3,1267,4,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22,5 x 18 cm",600,"Oct 25, 2020 9:00:45 PM",650,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none
4,1267,5,CRUZEIRO SEIXAS - NASC. 1920,Sem título,"técnica mista sobre papel, assinada","Dim. - 22 x 18,5 cm",600,"Oct 25, 2020 9:01:00 PM",850,https://leiloescml-aclserviatildese.netdna-ssl...,NaN,1,cruzeiro seixas,none,none


Let's start by dealing with the dates of artworks with only 1 author.

In [115]:
df_cml[df_cml['Dates'].isna()]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,Dates
1867,1142,147,LUÍS OSÓRIO,Sem título,"óleo sobre platex, assinado e datado de 93",Dim. - 42 x 32 cm,40,"Dec 30, 2018 9:36:30 PM",40,https://leiloescml-aclserviatildese.netdna-ssl...,1,luís osório,none,none,NaN


We have a missing value and it corresponds to an author that has no Birth/Death rate assigned.

In [117]:
df_cml[(df_cml['1 Author Name'] == 'luís osório')]

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,Dates
1867,1142,147,LUÍS OSÓRIO,Sem título,"óleo sobre platex, assinado e datado de 93",Dim. - 42 x 32 cm,40,"Dec 30, 2018 9:36:30 PM",40,https://leiloescml-aclserviatildese.netdna-ssl...,1,luís osório,none,none,NaN


We should later remove punctuation but for now all combinations were tested and only 1 Luís Osório was found. Therefore we cannot use info from another row to fill in this one and therefore  will replace it with "No Information".

In [118]:
df_cml.loc[df_cml['Dates'].isna(),'Dates'] = 'no information'

In [119]:
filt = df_cml['Number of Authors'] == 1
tmp = df_cml[filt]['Dates'].str.split(',')

In [120]:
#Check if we have cases with more than 2 dates and only 1 author:
tmp[tmp.apply(lambda x: len(x))>2]

Series([], Name: Dates, dtype: object)

Good, it seems at most we have 2 dates per author which is what is expected. We can now select the first date for date of Birth and the second for Date of Death.

In [121]:
#tmp[tmp.apply(lambda x: len(x))==2]

In [122]:
#Assign the author Date of Birth
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(lambda x: x[0])

#Assign the author Date of Death
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(lambda x: x[1] if len(x)==2 else 'no information')

Now, onwards to the cases with 2 or more authors.

In [123]:
filt = df_cml['Number of Authors'] >= 2
tmp = df_cml[filt]['Dates'].str.split(',')

Yikes! Here we need to be careful as we have more cases of dates of birth than death and we need to identify them. nasc. Is a good indicator word, otherwise, we can use the fact that some dates are too recent to be date of birth and vice-versa.

In [124]:
def choose_date(x):
    """
    This function was created to be used with pandas apply. It receives a list and will extract the dates from it and infer which are date of birth and which are dates of death.
    
    Returns the list of dates list_dates.
    """
    len_list = len(x)
    list_dates = []
    counter = 1
        
    for i in range(len_list):
        if i == 0:
            list_dates.append(int(re.findall(r'\d+',x[i])[0]))
            counter += 1
        else:
            if 'nasc' in x[i] and counter % 2 != 0:
                list_dates.append(int(re.findall(r'\d+',x[i])[0]))
                counter += 1   
            elif 'nasc' in x[i] and counter % 2 == 0:
                list_dates.append('no information')
                counter += 1
                list_dates.append(int(re.findall(r'\d+',x[i])[0]))
                counter += 1 
            elif int(re.findall(r'\d+',x[i])[0]) < list_dates[counter-2] + 40 and counter % 2 == 0:
                list_dates.append('no information')
                counter += 1
                list_dates.append(int(re.findall(r'\d+',x[i])[0]))
                counter += 1
            else:
                list_dates.append(int(re.findall(r'\d+',x[i])[0]))
                counter += 1
    if (counter-1) % 2 != 0:
        list_dates.append('no information')
    return list_dates

In [125]:
#For cases with 2 authors:
filt = df_cml['Number of Authors'] == 2
#Assign the First author Date of Birth
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[0]))
#Assign the First author Date of Death
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[1]))
#Assign the Second author Date of Birth
df_cml.loc[filt,'Second Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[2]))
#Assign the Second author Date of Death
df_cml.loc[filt,'Second Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[3]))

#For cases with 3 authors:
filt = df_cml['Number of Authors'] == 3
#Assign the First author Date of Birth
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[0]))
#Assign the First author Date of Death
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[1]))
#Assign the Second author Date of Birth
df_cml.loc[filt,'Second Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[2]))
#Assign the Second author Date of Death
df_cml.loc[filt,'Second Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[3]))
#Assign the Third author Date of Birth
df_cml.loc[filt,'Third Author Birth'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[4]))
#Assign the Third author Date of Death
df_cml.loc[filt,'Third Author Death'] = df_cml[filt]['Dates'].str.split(',').apply(choose_date).apply(lambda x: str(x[5]))

Great! Now for the final cleanup.

In [126]:
#If only 1 author exists
filt = df_cml['Number of Authors'] == 1
#Fill NA with "No Information"
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['First Author Birth'].fillna('no information')
df_cml.loc[filt,'Second Author Birth'] = df_cml[filt]['Second Author Birth'].fillna('not applicable')
df_cml.loc[filt,'Third Author Birth'] = df_cml[filt]['Third Author Birth'].fillna('not applicable')
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['First Author Death'].fillna('no information')
df_cml.loc[filt,'Second Author Death'] = df_cml[filt]['Second Author Death'].fillna('not applicable')
df_cml.loc[filt,'Third Author Death'] = df_cml[filt]['Third Author Death'].fillna('not applicable')
#If only 2 authors exists
filt = df_cml['Number of Authors'] == 2
#Fill NA with "No Information"
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['First Author Birth'].fillna('no information')
df_cml.loc[filt,'Second Author Birth'] = df_cml[filt]['Second Author Birth'].fillna('no information')
df_cml.loc[filt,'Third Author Birth'] = df_cml[filt]['Third Author Birth'].fillna('not applicable')
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['First Author Death'].fillna('no information')
df_cml.loc[filt,'Second Author Death'] = df_cml[filt]['Second Author Death'].fillna('no information')
df_cml.loc[filt,'Third Author Death'] = df_cml[filt]['Third Author Death'].fillna('not applicable')
#If only 3 authors exists
filt = df_cml['Number of Authors'] == 3
#Fill NA with "No Information"
df_cml.loc[filt,'First Author Birth'] = df_cml[filt]['First Author Birth'].fillna('no information')
df_cml.loc[filt,'Second Author Birth'] = df_cml[filt]['Second Author Birth'].fillna('no information')
df_cml.loc[filt,'Third Author Birth'] = df_cml[filt]['Third Author Birth'].fillna('no information')
df_cml.loc[filt,'First Author Death'] = df_cml[filt]['First Author Death'].fillna('no information')
df_cml.loc[filt,'Second Author Death'] = df_cml[filt]['Second Author Death'].fillna('no information')
df_cml.loc[filt,'Third Author Death'] = df_cml[filt]['Third Author Death'].fillna('no information')

In [127]:
#Make sure all entries are strings
df_cml['First Author Birth'] = df_cml['First Author Birth'].astype('str')
df_cml['Second Author Birth'] = df_cml['Second Author Birth'].astype('str')
df_cml['Third Author Birth'] = df_cml['Third Author Birth'].astype('str')
df_cml['First Author Death'] = df_cml['First Author Death'].astype('str')
df_cml['Second Author Death'] = df_cml['Second Author Death'].astype('str')
df_cml['Third Author Death'] = df_cml['Third Author Death'].astype('str')

In [128]:
df_cml['First Author Birth'] = df_cml['First Author Birth'].str.replace('nasc. ','')
df_cml['Second Author Birth'] = df_cml['Second Author Birth'].str.replace('nasc. ','')
df_cml['Third Author Birth'] = df_cml['Third Author Birth'].str.replace('nasc. ','')

After some exploration, some odd dates can be found:

- 19;
- 197.

In [129]:
df_cml[df_cml['First Author Birth'] == '19']

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,Dates,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death
1868,1142,148,JOÃO MÁRIO PENICHEIRO - 19??-2012,Sem título,"acrílico sobre tela, assinado e datado de 1964","Dim. - 48,5 x 57,5 cm",150,"Dec 30, 2018 9:36:45 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,joão mário penicheiro??,none,none,"19,2012",19,2012,not applicable,not applicable,not applicable,not applicable


This seems to be the equivalent of séc. XX (XX century) as the date is not known.

In [130]:
df_cml.loc[df_cml['First Author Birth'] == '19','First Author Birth'] = 'séc. xx'

In [131]:
df_cml[df_cml['First Author Birth'] == '197']

,Auction Number,Artwork Number,Author,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,Dates,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death
2730,1084,143,NUNO MEDEIROS - NASC. 197O,"""A águia de Pathemos""","óleo sobre tela, pequenos defeitos, assinado e...","Dim. - 65,5 x 54 cm",100,"Mar 19, 2018 9:35:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,nuno medeiroso,none,none,nasc. 197,197,no information,not applicable,not applicable,not applicable,not applicable
3234,1047,112,NUNO MEDEIROS - NASC. 197O,"""A águia de Pathemos""","óleo sobre tela, pequenos defeitos, assinado e...","Dim. - 65,5 x 54 cm",200,"Sep 10, 2017 9:28:00 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,nuno medeiroso,none,none,nasc. 197,197,no information,not applicable,not applicable,not applicable,not applicable


Here we can see that an o was used instead of 0 causing the problem. Let's manually correct it.

In [132]:
df_cml.loc[df_cml['First Author Birth'] == '197','First Author Birth'] = '1970'

#### Drop Author and Dates Columns

We can now drop the Author and Dates columns.

In [133]:
df_cml = df_cml.drop(['Author','Dates'],axis=1)

## Title

In [134]:
df_cml.head(1)

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death
0,1222,1,Sem título,"par de técnicas mistas sobre papel, assinadas",Dim. - 28 x 37 cm,40,"Feb 2, 2020 9:00:00 PM",60,https://leiloescml-aclserviatildese.netdna-ssl...,1,horta e costa,none,none,1953,no information,not applicable,not applicable,not applicable,not applicable


## Technique

Let's explore the text to discover how to best approach the data in this column.

We want to capture techniques or materials that may be relevant for the sale price! Therefore, we should strip the text of punctuation and get a wordcount as well as ngram count to try and identify words and expressions that can be captured.

Let's exclude portuguese stopwords, using the words listed in https://github.com/stopwords-iso/stopwords-pt.

In [135]:
#Let's obtain a list of stopwords, without punctuation.
list_stopwords = pd.read_csv('stopwords-pt.txt',header=None)[0].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').tolist()

#Let's add some words to the list of stopwords.
list_stopwords.append('paragrafo')
list_stopwords.append('sec')
list_stopwords.append('xxi')
list_stopwords.append('xx')
list_stopwords.append('xix')
list_stopwords.append('xx/xxi')
list_stopwords.append('xix/xx')
list_stopwords.append('-')
list_stopwords.append('')
list_stopwords.append('xxxxi')
list_stopwords.append('xixxx')

In [136]:
#Removing punctuation
df_cml['Technique'] = df_cml['Technique'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

#Remove certain patterns
df_cml['Technique Preprocessed'] = df_cml['Technique'].str.replace(',','').str.replace('(','').str.replace(')','').str.replace('\d+','').str.replace(r'/','').str.replace(r'-','').str.replace('.','').str.replace('?','')

#Tokenize
df_cml['Technique Tokenized'] = df_cml['Technique Preprocessed'].str.split(' ')

#Remove stopwords
df_cml['Technique Tokenized'] = df_cml['Technique Tokenized'].apply(lambda x: [word for word in x if word not in list_stopwords])

#Lemmatization
#Store the technique as a list of phrases
list_phrases = df_cml['Technique Tokenized'].apply(lambda x: ' '.join(x)).tolist()

#Call the NLP Pipeline
option = {"tokenizer" : True,
            "pos_tagger" : True,
            "lemmatizer" : True,
            "entity_recognition" : False,
            "np_chunking" : True,
            "pre_load" : True,
            "string_or_array" : True
}
text = new_full_pipe(list_phrases,options=option)
#text.print_conll()

#Let's make a list of lists from the outut of the lemmatizer
def group_lists(list_words, sep):
    group = []
    for element in list_words:
        if element == sep:
            yield group
            group = []
        else: 
            group.append(element)
    if group != []:
        yield group

result = list(group_lists(text.lemas, 'EOS'))

#Store lemmas on the dataframe
df_cml['Technique Lemmatized'] = result

Let's get the top 100 most frequent words __without__ Lemmatization

In [137]:
#word_count_technique - Top 100
word_count = Counter(df_cml['Technique Tokenized'].str.join(' ').str.cat(sep=' ').split(' ')).most_common(200)
#word_count

all_counts_tokenized = dict()
all_counts_tokenized[1] = {tuple_word[0]:tuple_word[1] for tuple_word in word_count}
for size in 2, 3, 4, 5:
    tmp = FreqDist(ngrams(df_cml['Technique Tokenized'].apply(lambda x: ' '.join(x) + " ").str.cat(sep=' paragrafo ').split(' '), size))
    #Exclude the tuples that include 'paragrafo'
    list_tuples = [tuple_words for tuple_words in list(tmp.keys()) if not any(word == 'paragrafo' for word in tuple_words)]
    tmp_dict = {key:tmp[key] for key in list_tuples}
    #Sort the dictionary
    all_counts_tokenized[size] = {k: v for k, v in sorted(tmp_dict.items(), key=lambda item: item[1],reverse=True)}

Let's get the top 100 most frequent words __with__ Lemmatization

In [138]:
#word_count_technique - Top 100
word_count = Counter(df_cml['Technique Lemmatized'].str.join(' ').str.cat(sep=' ').split(' ')).most_common(200)
#word_count

all_counts_lemmatized = dict()
all_counts_lemmatized[1] = {tuple_word[0]:tuple_word[1] for tuple_word in word_count}
for size in 2, 3, 4, 5:
    tmp = FreqDist(ngrams(df_cml['Technique Lemmatized'].apply(lambda x: ' '.join(x) + " ").str.cat(sep=' paragrafo ').split(' '), size))
    #Exclude the tuples that include 'paragrafo'
    list_tuples = [tuple_words for tuple_words in list(tmp.keys()) if not any(word == 'paragrafo' for word in tuple_words)]
    tmp_dict = {key:tmp[key] for key in list_tuples}
    #Sort the dictionary
    all_counts_lemmatized[size] = {k: v for k, v in sorted(tmp_dict.items(), key=lambda item: item[1],reverse=True)}

Ok, now we have n grams of the words used on the Technique column! This should help us identify the most important terms and expressions that need to be captured (One Hot Encoded).

Lemmas appear to be mostly well obtained with few exceptions. Regardless we'll test the model's performance with and without lemmatization to gauge it's importance.

Let's OHE the Lemmas!

In [139]:
#OHE the lemma words
for i in list(all_counts_lemmatized[1].keys()):
    df_cml[i] = 0
    df_cml.loc[df_cml['Technique Lemmatized'].apply(lambda x: i in x),i] = 1

In [140]:
df_cml.head(2)

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,Technique Preprocessed,Technique Tokenized,Technique Lemmatized,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca
0,1222,1,Sem título,"par de tecnicas mistas sobre papel, assinadas",Dim. - 28 x 37 cm,40,"Feb 2, 2020 9:00:00 PM",60,https://leiloescml-aclserviatildese.netdna-ssl...,1,horta e costa,none,none,1953,no information,not applicable,not applicable,not applicable,not applicable,par de tecnicas mistas sobre papel assinadas,"[par, tecnicas, mistas, papel, assinadas]","[par, tecnico, mista, papel, assinado]",0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1222,2,"""Arco da Rua Augusta""","oleo sobre tela, assinado barros e datado de 2000",Dim. - 60 x 50 cm,300,"Feb 2, 2020 9:00:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,oleo sobre tela assinado barros e datado de,"[oleo, tela, assinado, barros, datado]","[oleo, tela, assinar, barro, datado]",1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [141]:
#Drop the columns "Technique Preprocessd, Tokenized and Lemmatized"
df_cml = df_cml.drop(['Technique Tokenized','Technique Preprocessed','Technique Lemmatized'],axis=1).copy() #Technique
df_cml.head()

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca
0,1222,1,Sem título,"par de tecnicas mistas sobre papel, assinadas",Dim. - 28 x 37 cm,40,"Feb 2, 2020 9:00:00 PM",60,https://leiloescml-aclserviatildese.netdna-ssl...,1,horta e costa,none,none,1953,no information,not applicable,not applicable,not applicable,not applicable,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1222,2,"""Arco da Rua Augusta""","oleo sobre tela, assinado barros e datado de 2000",Dim. - 60 x 50 cm,300,"Feb 2, 2020 9:00:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1222,3,Sem título,"oleo sobre tela, assinado e datado de 1986",Dim. - 100 x 82 cm,300,"Feb 2, 2020 9:03:26 PM",400,https://leiloescml-aclserviatildese.netdna-ssl...,1,rui cunha,none,none,1942,no information,not applicable,not applicable,not applicable,not applicable,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1222,4,"""Coimbra""","oleo sobre platex, assinado",Dim. - 40 x 50 cm,150,"Feb 2, 2020 9:03:47 PM",600,https://leiloescml-aclserviatildese.netdna-ss

## Dimensions

In [142]:
#Initialize the new columns
df_cml['Dim 1'] = np.nan
df_cml['Dim 2'] = np.nan
df_cml['Dim 3'] = np.nan

In [143]:
df_cml[df_cml['Dimensions'].str.contains('Dim')==False]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3
287,1222,288,Vista de Lisboa a partir do Jardim de São Pedr...,"aguarela sobre papel, assinada e datada de 1975",Vista de Lisboa a partir do Jardim de São Pedr...,50,"Feb 2, 2020 10:11:45 PM",60,https://leiloescml-aclserviatildese.netdna-ssl...,1,jaroslaw gruzewski,none,none,1958,no information,not applicable,not applicable,not applicable,not applicable,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
514,1218,58,"""Jardim da Graça""","serigrafia sobre papel, assinada e datada de 1...","""Jardim da Graça""",20,"Jan 12, 2020 9:14:15 PM",38,https://leiloescml-aclserviatildese.netdna-ssl...,1,ernani oliveira,none,none,1936,no information,not applicable,not applicable,not applicable,not applicable,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1188,1183,114,"""O jardim encantado""","serigrafia sobre papel, assinado e datado de 1...","""O jardim encantado""",80,"Jul 28, 2019 9:28:15 PM",100,https://leiloescml-aclserviatildese.netdna-ssl...,1,bartolomeu cid dos santos,none,none,1931,2008,not applicable,not applicable,not applicable,not applicable,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

A small number of entries aren't dimensions but rather the title! These will be changed to "No information".

In [144]:
df_cml.loc[df_cml['Dimensions'].str.contains('Dim')==False,'Dimensions'] = 'no information'

In [145]:
df_cml['Dimensions'].value_counts().head(10)

Dim. - 14 x 14 cm      59
Dim. - 70 x 50 cm      23
Dim. - 38,5 x 59 cm    22
Dim. - 100 x 100 cm    20
Dim. - 65 x 50 cm      19
Dim. - 100 x 70 cm     17
Dim. - 32 x 32 cm      16
Dim. - 50 x 70 cm      16
Dim. - 25 cm           16
Dim. - 24 cm           15
Name: Dimensions, dtype: int64

We have rows with 1, 2,3 or more dimensions (the lattercase are probably multiple items on a single auction).

We also have reference to the unit (cm), but we should be aware that other units may be used (mm, m, etc.).

In [146]:
#There are some Missing Values. Let's replace them with no information
df_cml[df_cml['Dimensions'].isna()]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3
77,1222,78,Sem título,"serigrafia sobre papel, assinada e datada de 1...",NaN,200,"Feb 2, 2020 9:19:15 PM",575,https://leiloescml-aclserviatildese.netdna-ssl...,1,manuel cargaleiro,none,none,1927,no information,not applicable,not applicable,not applicable,not applicable,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
328,1222,329,Sem título,"tecnica mista sobre papel, assinada, dedicada ...",NaN,400,"Feb 2, 2020 10:22:00 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,graça morais,none,none,1948,no information,not applicable,not applicable,not applicable,not applicable,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
418,1222,419,Sem título,"gravura sobre papel, picos de acidez, defeito,...",NaN,30,"Feb 2, 2020 10:44:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
454,1222,455,Sem título,"tecnica mista sobre tela, assinado",NaN,600,"Feb 2, 2020 10:53:30 PM",600,https://leiloescml

In [147]:
#These are missing values indeed!
df_cml['Dimensions'] = df_cml['Dimensions'].fillna('no information')

#Replace , with .
df_cml['Dimensions'] = df_cml['Dimensions'].str.replace(', ','.').str.replace(',','.')

Let's extract the numbers from the rows, as well as the units.

In [148]:
#Extract list with dimensions
df_cml['Dimension List'] = df_cml['Dimensions'].str.findall(r'(\d+\,+\d*|\d+\.+\d*|\d+|no information)')
#Extract number of dimensions
df_cml['Number of Dimensions'] = df_cml['Dimension List'].apply(lambda x: len(x))
#Extract unit of dimensions
df_cml['Dimension Units'] = df_cml['Dimensions'].str.findall(r'(no information)$|(cm)$|(mm)$').apply(lambda x: x[0][0] if x[0][0] != '' else (x[0][1] if x[0][1] != '' else x[0][2]))

Now let's see how many artworks exist for the various numbers of dimensions.

In [149]:
df_cml['Number of Dimensions'].value_counts()

2    3150
1     552
3     264
4      15
6       8
5       3
8       1
Name: Number of Dimensions, dtype: int64

There are some atworks with 4 or more dimensions. These must be actions with more than 1 artwork. But how to deal with those?

And what about artworks with only 1 dimension?

In [150]:
df_cml[df_cml['Number of Dimensions'] == 1].sample(4)

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units
2512,1098,174,"Espelho ""Sol""","moldura em folha de metal dourada, espelho bis...",Dim. - 127 cm,100,"May 20, 2018 9:43:30 PM",100,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,[127],1,cm
2817,1084,230,Lança,"ferro, cabo entalhada com aplicacao em pele, a...",Dim. - 134 cm,80,"Mar 19, 2018 9:57:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,[134],1,cm
1578,1170,158,Jarra,"vidro moldado e gravado, faixa gravada e doura...",Dim. - 20 cm,50,"May 19, 2019 9:39:15 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,N

These seem to be a combination of cases where no information being available (0 would be a more correct number of dimensions) and some artworks which only have 1 dimension.

In [151]:
#Replace number of dimensions when 'no information' from 1 to 0
df_cml.loc[(df_cml['Number of Dimensions'] == 1) & (df_cml['Dimensions'].str.lower() == 'no information'),'Number of Dimensions'] = 0

We can now try to extract the number of artworks.

In [152]:
df_cml['Number of Artworks'] = 0

list_words_nbr = ['^dois |^duas |^par |^2 ',
                  '^tres |^três |^3 ',
                  '^quatro |^4 ',
                  '^cinco |^5 ',
                  '^seis |^6 ',
                  '^sete |^7 ',
                  '^oito |^8 ',
                  '^nove |^9 ',
                  '^dez |^10 ','^onze |^11 ','^doze |^12 ','^treze |^13 ','^catorze |^14 ','^quinze |^15 ','^dezasseis |^16 ','^dezassete |^17 ','^dezoito |^18 ','^dezanove |^19 ',
                 '^vinte |^20 ','^vinte e um |vinte um |^21 ','^vinte e dois |vinte dois |^22 ','^vinte e tres |vinte e três |vinte tres |vinte três |^23 ','^vinte e quatro |vinte quatro |^24 ',
                 '^vinte e cinco |vinte cinco |^25 ','^vinte e seis |vinte seis |^26 ','^vinte e sete |vinte sete |^27 ','^vinte e oito |vinte oito |^28 ','^vinte e nove |vinte nove |^29 ',
                 '^trinta |^30 ','^quarenta |^40 ','^cinquenta |^50 ','^sexta |^60 ','^setenta |^70 ','^oitenta |^80 ','^noventa |^90 ','^cem |^100 ']
counter = 2
for i in list_words_nbr:
    df_cml.loc[(df_cml['Title'].str.lower().str.contains(i)) | (df_cml['Technique'].str.lower().str.contains(i)),'Number of Artworks'] = counter
    counter += 1
    
#Correct the number of artworks for cases where multiple artworks cannot be identified via title or technique, but with dimensions information.
df_cml.loc[(df_cml['Dimensions'] != 'no information') & (df_cml['Number of Artworks'] == 0),'Number of Artworks'] = 1

In [153]:
#df_cml[df_cml['Number of Artworks'] == 1]
df_cml['Number of Artworks'].value_counts()

1     3720
2      159
0       31
3       29
4       28
6        8
5        4
27       2
11       2
9        2
7        2
14       2
8        2
13       1
12       1
Name: Number of Artworks, dtype: int64

Now, data shows that in some cases, 1 artwork has 2 or more sets of dimensions (for example the total size of the canvas and the size of the painted area of the canvas).

We also have sculptures which are volumetric as well as artworks that only have information on one of the dimensions. 

This presents a challenge that needs to be addressed.

### With 1 Artwork

In [154]:
df_cml[(df_cml['Number of Artworks'] == 1) & (df_cml['Number of Dimensions'] > 3)]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks
812,1199,38,Sem título,"serigrafia sobre papel, assinada e datada de 1...",Dim. - 49 x 66.5 (frente); 27.5 x 50 (mancha) cm,40,"Nov 3, 2019 9:20:05 PM",130,https://leiloescml-aclserviatildese.netdna-ssl...,1,tomthomaz de mello,none,none,1906,1990,not applicable,not applicable,not applicable,not applicable,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"[49, 66.5, 27.5, 50]",4,cm,1
813,1199,39,Sem título,"serigrafia sobre papel, assinada e datada de 1...",Dim. - 47.5 x 66 (frente) ; 27.5 x 51 cm (manc...,40,"Nov 3, 2019 9:20:29 PM",110,https://leiloescml-aclserviatildese.netdna-ssl...,1,tomthomaz de mello,none,none,1906,1990,not applicable,not applicable,not applicable,not applicable,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"[47.5, 66, 27.5, 51]",4,cm,1
814,1199,40,Sem título,"serigrafia sobre papel, assinada e datada de 1...",Dim. - 48 x 67 (frente); 27.5 x 51 (mancha) cm,40,"Nov 3, 2019 9:24:46 PM",120,https://leiloescml-aclserviatildese.netdna-ssl...,1,tomthomaz de mello,none,none,1906,1990,not applicable,not applicable,not applicable,not applicable,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Not many cases exist of single artworks with more than 3 dimensions. We'll have to decide a criteria for these cases.

It seems that most of these cases falls into 1 of 2 categories: 

1 - They are paintings that are detailing the size of the painted area as well as the total "paper" area.

2 - They are a set of objects with the dimensions specified for them.

In all cases, a ; or / divides them and we can use this to our advantage. Also for paintings (we'll look for serigrafia, tela, oleo etc.), we will consider the size of the smallest dimensions as those are the dimensions of interest. For 2, we'll consider only the first 3 dimensions.

In [155]:
# For 1:
filt = ((df_cml['serigrafia'] == True) | (df_cml['oleo'] == True)| (df_cml['tela'] == True)| (df_cml['papel'] == True)) & ((df_cml['Number of Artworks'] == 1) & (df_cml['Number of Dimensions'] > 3))
df_cml.loc[filt,'Dimension List'] = df_cml[filt]['Dimension List'].apply(lambda x: x[:2] if float(x[0])*float(x[1]) < float(x[2])*float(x[3]) else x[2:])

# For 2:
filt = (df_cml['Number of Artworks'] == 1) & (df_cml['Number of Dimensions'] > 3) & (df_cml['serigrafia'] == False) & (df_cml['oleo'] == False) & (df_cml['tela'] == False)& (df_cml['papel'] == False)
df_cml.loc[filt,'Dimension List'] = df_cml[filt]['Dimension List'].apply(lambda x: x[:3])

### With Multiple Artworks

In [156]:
filt = ((df_cml['Number of Artworks'] > 1) & (df_cml['Number of Dimensions'] > 4))
df_cml[filt].sample(4)

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks
2813,1084,226,"Duas máscaras femininas ""Mwana Pwo""","madeira esculpida e patinada, tchokwe (quioco)...",Dim. - 15 x 11 x 11 cm e 14 x 11.5 x 8 cm,120,"Mar 19, 2018 9:58:23 PM",120,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"[15, 11, 11, 14, 11.5, 8]",6,cm,2
3866,1014,60,Três paisagens,"tecnicas mistas sobre papel, assinadas e datad...",Dim. - 23.5 x 175; 21 x 230; 32 x 292 cm,30,"Feb 5, 2017 9:14:45 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,cervantes de haro,none,none,séc. xix,no information,not applicable,not applicable,not applicable,not applicable,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"[23.5, 175, 21, 230, 32, 292]",6,cm,3
3276,1047,154,Dois apliques,"ferro pintado, portugueses, sec. xx (anos 50),...",Dim. - 21 x 27 x 12 e 17 x 27 x 12 cm,10,"Sep 10, 2017 9:38:30 PM",Not sold,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [157]:
filt = df_cml['Number of Dimensions'] > 3
df_cml[filt][((df_cml[filt]['Number of Dimensions'] / df_cml[filt]['Number of Artworks']) % 1) > 0 ]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks
1071,1199,297,Par de porta-retratos,"art deco, metal cromado, portugueses, sec. xx,...",Dim. - (total) 17 x 13 x 4 cm; (fotografia) 14...,30,"Nov 3, 2019 10:14:00 PM",36,https://leiloescml-aclserviatildese.netdna-ssl...,1,no information,none,none,no information,no information,not applicable,not applicable,not applicable,not applicable,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"[17, 13, 4, 14, 8.5]",5,cm,2


So for multiple artworks we have 2 main cases... When we have more dimensions than artworks and vice versa.

More dimensions than artworks:

    Even number of dimensions (> 3) - We can obtain N dimensions, N being Number of Dimensions / Number of Artworks.

    Uneven number of dimensions (> 3) - Only one instance of this exists with more than 3 dimensions. In this case, we will take the first 3 dimensions on the list. 
    
    Dimensions between 1 and 3 - Store all dimensions in Dim 1 to 3.
    
More artworks than dimensions:

    No cases exist with more artwoks than dimensions with more than 3 dimensions. Therefore, in this situation all dimensions will be stored in Dim 1 to 3.    

In [158]:
filt = (df_cml['Number of Artworks'] > df_cml['Number of Dimensions']) & (df_cml['Number of Dimensions'] > 3)
df_cml[filt]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks


We can now fill in Dim 1, Dim 2 and Dim 3!

When only 1 artwork exists:

In [159]:
filt = df_cml['Number of Artworks'] == 1
#Fill Dim 1
df_cml.loc[filt,'Dim 1'] = df_cml[filt]['Dimension List'].apply(lambda x: x[0] if len(x) >= 1 else np.nan)
#Fill Dim 2
df_cml.loc[filt,'Dim 2'] = df_cml[filt]['Dimension List'].apply(lambda x: x[1] if len(x) >= 2 else np.nan)
#Fill Dim 3
df_cml.loc[filt,'Dim 3'] = df_cml[filt]['Dimension List'].apply(lambda x: x[2] if len(x) >= 3 else np.nan)

With multiple artworks:

If there are 3 or less dimensions, we will consider those for Dim 1 to 3.

In [160]:
filt = (df_cml['Number of Artworks'] > 1) &  (df_cml['Number of Dimensions'] <= 3)
df_cml.loc[filt,'Dim 1'] = df_cml[filt]['Dimension List'].apply(lambda x: x[0] if len(x) >= 1 else np.nan)
df_cml.loc[filt,'Dim 2'] = df_cml[filt]['Dimension List'].apply(lambda x: x[1] if len(x) >= 2 else np.nan)
df_cml.loc[filt,'Dim 3'] = df_cml[filt]['Dimension List'].apply(lambda x: x[2] if len(x) >= 3 else np.nan)

If there are more than 3 dimensions then we follow the strategy outlined above.

In [161]:
filt = (df_cml['Number of Artworks'] > 1) & (df_cml['Number of Dimensions'] > 3) & ((df_cml['Number of Dimensions'] / df_cml['Number of Artworks']) % 1 > 0)
df_cml.loc[filt,'Dim 1'] = df_cml[filt]['Dimension List'].apply(lambda x: x[0] if len(x) >= 1 else np.nan)
df_cml.loc[filt,'Dim 2'] = df_cml[filt]['Dimension List'].apply(lambda x: x[1] if len(x) >= 2 else np.nan)
df_cml.loc[filt,'Dim 3'] = df_cml[filt]['Dimension List'].apply(lambda x: x[2] if len(x) >= 3 else np.nan)

In [162]:
#For this to work we need to select only a subset of the dimensions, so we'll choose the first set of dimensions.
filt = (df_cml['Number of Artworks'] > 1) & (df_cml['Number of Dimensions'] > 3) & ((df_cml['Number of Dimensions'] / df_cml['Number of Artworks']) % 1 == 0)
list_nbr_dimensions = (df_cml.loc[filt,'Number of Dimensions'] / df_cml.loc[filt,'Number of Artworks']).values.tolist()
idx = (df_cml.loc[filt,'Number of Dimensions'] / df_cml.loc[filt,'Number of Artworks']).index

for i in range(len(idx)):
    for j in range(int(list_nbr_dimensions[i])):
        df_cml.loc[idx[i],'Dim '+ str(j+1)] = df_cml.loc[idx[i],'Dimension List'][j]

## Estimated Price

In [163]:
df_cml[df_cml['Estimated Price'].isna()]

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks


In [164]:
df_cml['Estimated Price'].describe()

count    3993.000000
mean      107.236915
std       117.366878
min         1.000000
25%        30.000000
50%        70.000000
75%       150.000000
max       900.000000
Name: Estimated Price, dtype: float64

No processing needed out of the bat!

## Date of Auction End

In [165]:
#Convert te Date of Auction End to datetime format
df_cml['Date of Auction End'] = pd.to_datetime(df_cml['Date of Auction End'])

#Create new columns with Year, Month, Week, Day, Day of Week, Hour
df_cml['Year'] = df_cml['Date of Auction End'].dt.year
df_cml['Month'] = df_cml['Date of Auction End'].dt.month
df_cml['Day'] = df_cml['Date of Auction End'].dt.day
df_cml['Day of Week'] = df_cml['Date of Auction End'].dt.day_name()
df_cml['Hour'] = df_cml['Date of Auction End'].dt.hour

## Sale Price

In [166]:
df_cml['Sale Price'].unique()

array(['60', 'Not sold', '400', '600', '375', '180', '160', '14', '120',
       '46', '40', '26', '36', '1,050', '700', '325', '85', '28', '110',
       '30', '65', '24', '150', '80', '32', '48', '240', '70', '550',
       '130', '170', '55', '100', '22', '34', '260', '575', '95', '200',
       '42', '90', '350', '44', '190', '725', '38', '10', '18', '50',
       '20', '16', '220', '280', '75', '300', '450', '1,350', '525',
       '650', '250', '140', '900', '1,300', '475', '12', '500', '425',
       '1,700', '800', '2,200', '850', '360', '270', '625', '1,800',
       '4,000', '675', '3,500', '23', '290', '35', '15', '1,600', '105',
       '13', '385', '750', '3,750', '950', '775', '1,150', '1,200', '585',
       '2,900', '2,000', '335', '17', '1,000', '31', '9', '810', '1,100',
       '2,400', '2,100', '11', '33', '37', '29', '5', '125', '145',
       '2,500', '1,250', '21', '27', '25', '8', '4', '19', '7'],
      dtype=object)

So regarding sale price, we need to clean it up. Namely let's eliminate the , in the sale prices.

Let's also break this down into 2 target variables, one of Sold with 0 or 1 (not sold or sold) and another with Final Price (with 0 for Not sold and the remaining a value).

In [167]:
#Create the Sold feature
df_cml['Sold'] = (df_cml['Sale Price'] != 'Not sold').astype('uint8')

#Create the Final Price feature
df_cml['Final Price'] = df_cml['Sale Price'].str.replace(',','').apply(lambda x: 0 if x=='Not sold' else x)

# Dropping Old Columns

In [168]:
df_cml.head(1)

,Auction Number,Artwork Number,Title,Technique,Dimensions,Estimated Price,Date of Auction End,Sale Price,Image URL,Number of Authors,1 Author Name,2 Author Name,3 Author Name,First Author Birth,First Author Death,Second Author Birth,Second Author Death,Third Author Birth,Third Author Death,assinar,papel,datado,numerado,serigrafia,tecnico,marcar,tela,misturar,defeito,pequeno,oleo,sinal,europeu,uso,metal,decoracao,vidro,verso,acrilico,madeira,escultura,identificar,portuguÃªs,policromado,gravura,``,assinatura,aguarela,'',tinta,chino,fabricar,autor,pa,falta,italiano,cromado,portugue,material,azulejo,platex,mancha,cristal,dourar,pintado,prateado,plastico,hc,relevado,base,humidade,colagem,metade,ceramico,art,deco,moldado,cartao,faianca,edicao,pico,litografia,fotografia,viuva,lamego,porcelana,vidrado,acidez,esbeicadela,france,barro,prova,pastel,negro,tampo,azul,desgaste,mecanismo,modelo,suporte,estrutura,assinado,fosco,prato,guache,provavelmente,pintura,bronze,branco,partido,pe,francÃªs,ferro,moldura,alemao,marmore,verde,tecido,provar,ligeiro,:,colada,lisboa,necessitar,revisao,europa,pintada,estofado,quartel,dedicar,colorir,atlantil,alessi,placa,dourada,parcialmente,etiqueta,aco,arte,patinado,tom,bicolor,compor,p,jose,pasta,copo,nasc,grafite,artista,desenho,aplicacoe,emoldurado,vl,vermelho,sintetico,oxidacao,rotacoe,mista,vinco,amarelo,pele,autografado,ea,atelier,interior,original,terracota,decoracoe,inox,quebraluz,costa,assento,esculpido,colado,murano,loico,castanho,vinil,carvao,aparentemente,estampa,estilo,lapil,galeria,inscricao,gravar,americano,cobrir,africo,disco,mogno,datar,caixa,globo,sacavem,fabrico,laminado,escola,folheado,tiragem,tampa,chaveno,taca,meado,soprado,exotico,abatjour,colorido,rasgao,indicacao,antonio,folha,baquelite,aleluia,massa,aluminio,barcelÃ£o,teca,Dim 1,Dim 2,Dim 3,Dimension List,Number of Dimensions,Dimension Units,Number of Artworks,Year,Month,Day,Day of Week,Hour,Sold,Final Price
0,1222,1,Sem título,"par de tecnicas mistas sobre papel, assinadas",Dim. - 28 x 37 cm,40,2020-02-02 21:00:00,60,https://leiloescml-aclserviatildese.netdna-ssl...,1,horta e costa,none,none,1953,no information,not applicable,not applicable,not applicable,not applicable,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,37,NaN,"[28, 37]",2,cm,2,2020,2,2,Sunday,21,1,60


In [169]:
#df_cml = df_cml.drop(['Title','Technique','Dimensions','Estimated Price','Date of Auction End','Sale Price','Image URL'],axis=1)

In [170]:
#Let's save this into a new DataFrame
df_cml.to_csv('.\cleaned_data\cleaned_df_cml.csv')